#### Imports

In [16]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import requests

import sys
sys.path.append('../edgar_utils')

from params import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Accessing EDGAR DB

In [17]:
#create a request header using your email as the user agent
headers = {
    'User-Agent': EXAMPLE_EMAIL
}

#instantiate the sec url for the requests
sec_url = SEC_URL

In [19]:
#fetech all companies data
response = requests.get(sec_url, headers=headers)
response

<Response [200]>

In [32]:
#save response data
data = response.json()

## Fetch Company Metadata

In [35]:
# check 1st value // format
first_entry = data['0']
first_entry

{'cik_str': 789019, 'ticker': 'MSFT', 'title': 'MICROSOFT CORP'}

In [34]:
# parse cik without leading zeros
first_entry = data['0']['cik_str']
first_entry

789019

In [36]:
# dict to df
companies_df = pd.DataFrame.from_dict(data, orient='index')
companies_df

,cik_str,ticker,title
0,789019,MSFT,MICROSOFT CORP
1,320193,AAPL,Apple Inc.
2,1045810,NVDA,NVIDIA CORP
3,1018724,AMZN,AMAZON COM INC
4,1652044,GOOGL,Alphabet Inc.
...,...,...,...
10418,1969475,BAYAR,Bayview Acquisition Corp
10419,1978528,QETAR,Quetta Acquisition Corp
10420,1978528,QETAU,Quetta Acquisition Corp
10421,1975218,NETDW,Nabors Energy Transition Corp. II


In [38]:
#add leading zeros to CIK
''' 
########## is the entity’s 10-digit Central Index Key (CIK), 
including leading zeros.
'''

companies_df['cik_str'] = companies_df['cik_str'].astype(str).str.zfill(10)

In [39]:
companies_df.head(5)

,cik_str,ticker,title
0,0000789019,MSFT,MICROSOFT CORP
1,0000320193,AAPL,Apple Inc.
2,0001045810,NVDA,NVIDIA CORP
3,0001018724,AMZN,AMAZON COM INC
4,0001652044,GOOGL,Alphabet Inc.


## IBM CIK

In [89]:
#save ibm cik for query
ibm_data = companies_df.loc[companies_df['ticker'] == 'IBM']
ibm_cik = ibm_data['cik_str'][0]
ibm_cik

'0000051143'

In [90]:
filing_response = requests.get(
                    f'https://data.sec.gov/submissions/CIK{ibm_cik}.json',
                    headers=headers
                    )

In [134]:
metadata = filing_response.json()
metadata.keys()

dict_keys(['cik', 'entityType', 'sic', 'sicDescription', 'insiderTransactionForOwnerExists', 'insiderTransactionForIssuerExists', 'name', 'tickers', 'exchanges', 'ein', 'description', 'website', 'investorWebsite', 'category', 'fiscalYearEnd', 'stateOfIncorporation', 'stateOfIncorporationDescription', 'addresses', 'phone', 'flags', 'formerNames', 'filings'])

In [133]:
#check filings
metadata['filings'].keys()

dict_keys(['recent', 'files'])

In [94]:
metadata['filings']['files']

[{'name': 'CIK0000051143-submissions-001.json',
  'filingCount': 2011,
  'filingFrom': '2011-05-10',
  'filingTo': '2018-05-09'},
 {'name': 'CIK0000051143-submissions-002.json',
  'filingCount': 2011,
  'filingFrom': '2006-05-16',
  'filingTo': '2011-05-09'},
 {'name': 'CIK0000051143-submissions-003.json',
  'filingCount': 2000,
  'filingFrom': '2002-07-17',
  'filingTo': '2006-05-15'},
 {'name': 'CIK0000051143-submissions-004.json',
  'filingCount': 315,
  'filingFrom': '1994-03-10',
  'filingTo': '2002-07-16'}]

In [95]:
forms_data = metadata['filings']['recent']

In [96]:
#store all filings as df
company_forms = pd.DataFrame.from_dict(forms_data)

In [97]:
company_forms.columns

Index(['accessionNumber', 'filingDate', 'reportDate', 'acceptanceDateTime',
       'act', 'form', 'fileNumber', 'filmNumber', 'items', 'size', 'isXBRL',
       'isInlineXBRL', 'primaryDocument', 'primaryDocDescription'],
      dtype='object')

In [98]:
columns = ['accessionNumber', 'reportDate',  'form']
company_forms[columns]

,accessionNumber,reportDate,form
0,0000051143-24-000015,2024-03-18,8-K
1,0001214659-24-004360,,PX14A6G
2,0001104659-24-032653,2023-12-31,ARS
3,0001104659-24-032647,,DEFA14A
4,0001104659-24-032641,2024-04-30,DEF 14A
...,...,...,...
1000,0001562180-18-002438,2018-05-09,4
1001,0001562180-18-002437,2018-05-09,4
1002,0001562180-18-002436,2018-05-09,4
1003,0001562180-18-002435,2018-05-09,4


In [99]:
form_10k = company_forms.loc[company_forms['form'] == '10-K']

In [100]:
form_10k

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
5,0000051143-24-000012,2024-02-26,2023-12-31,2024-02-26T06:41:57.000Z,34,10-K,001-02360,24673497,,79490429,1,1,ibm-20231231.htm,10-K
129,0001558370-23-002376,2023-02-28,2022-12-31,2023-02-28T16:26:11.000Z,34,10-K,001-02360,23685888,,53972096,1,1,ibm-20221231x10k.htm,10-K
289,0001558370-22-001584,2022-02-22,2021-12-31,2022-02-22T16:07:29.000Z,34,10-K,001-02360,22658090,,54190023,1,1,ibm-20211231x10k.htm,10-K
456,0001558370-21-001489,2021-02-23,2020-12-31,2021-02-23T17:08:23.000Z,34,10-K,001-02360,21666675,,52973273,1,1,ibm-20201231x10k.htm,10-K
638,0001558370-20-001334,2020-02-25,2019-12-31,2020-02-25T16:54:20.000Z,34,10-K,001-02360,20651634,,44890558,1,1,ibm-20191231x10k2af531.htm,10-K
844,0001047469-19-000712,2019-02-26,2018-12-31,2019-02-26T16:27:59.000Z,34,10-K,001-02360,19633815,,40132688,1,0,a2237254z10-k.htm,10-K


## Fetching company facts

In [102]:
#fetch the request
ibm_facts_response = requests.get(f'https://data.sec.gov/api/xbrl/companyfacts/CIK{ibm_cik}.json',
                         headers=headers)
ibm_facts_response

<Response [200]>

In [103]:
ibm_facts_data = ibm_facts_response.json()

In [109]:
ibm_facts_data.keys()

dict_keys(['cik', 'entityName', 'facts'])

In [127]:
ibm_facts = ibm_facts_data['facts']

key descriptions:
- dei - shares <br/>
- us-gaap  -  statements

### DEI

In [128]:
ibm_dei = ibm_facts['dei']['EntityCommonStockSharesOutstanding']
dei_label = ibm_dei['label']
dei_description = ibm_dei['description']

In [191]:
print(f'label:{dei_label} \n\ndescription:{dei_description}')

label:Entity Common Stock, Shares Outstanding 

description:Indicate number of shares or other units outstanding of each of registrant's classes of capital or common stock or other ownership interests, if and as stated on cover of related periodic report. Where multiple classes or units exist define each class/interest by adding class of stock items such as Common Class A [Member], Common Class B [Member] or Partnership Interest [Member] onto the Instrument [Domain] of the Entity Listings, Instrument.


In [137]:
ibm_shares = ibm_dei['units']['shares']
ibm_shares_df = pd.DataFrame.from_dict(ibm_shares)

In [138]:
ibm_shares_df.head(3)

,end,val,accn,fy,fp,form,filed,frame
0,2009-06-30,1310883577,0001104659-09-045198,2009,Q2,10-Q,2009-07-28,CY2009Q2I
1,2009-09-30,1313602936,0001104659-09-060554,2009,Q3,10-Q,2009-10-27,CY2009Q3I
2,2010-02-10,1299003390,0001047469-10-001151,2009,FY,10-K,2010-02-23,CY2009Q4I


### US-GAAP

In [165]:
ibm_gaap = ibm_facts['us-gaap']

In [166]:
ibm_acc_items = pd.DataFrame.from_dict(ibm_gaap).transpose()

In [172]:
#list of account names used in the statements
item_names = ibm_gaap.keys()

In [180]:
#list comprehension to look for a specifi account item
[item for item in item_names if 'Revenue' in item]

['ContractWithCustomerLiabilityRevenueRecognized',
 'CostOfRevenue',
 'DeferredRevenueCurrent',
 'DeferredRevenueNoncurrent',
 'FinancialServicesRevenue',
 'RevenueRemainingPerformanceObligation',
 'Revenues',
 'SalesRevenueGoodsNet',
 'SalesRevenueServicesNet',
 'SalesTypeLeaseRevenue',
 'SegmentReportingInformationIntersegmentRevenue',
 'SegmentReportingInformationRevenue',
 'SegmentReportingInformationRevenueFromExternalCustomers',
 'SegmentReportingReconcilingItemsRevenue']

#### Exploration: Revenues

In [188]:
rev_label = ibm_gaap['Revenues']['label']
rev_description = ibm_gaap['Revenues']['description']

In [190]:
print(f'label:{rev_label} \n\ndescription:{rev_description}')

label:Revenues 

description:Amount of revenue recognized from goods sold, services rendered, insurance premiums, or other activities that constitute an earning process. Includes, but is not limited to, investment and interest income before deduction of interest expense when recognized as a component of revenue, and sales and trading gain (loss).


In [214]:
#check entry format 
rev_entry = ibm_gaap['Revenues']['units']['USD']
revenues_df = pd.DataFrame.from_dict(rev_entry)

In [216]:
revenues_df.shape

(199, 9)

In [219]:
revenues_df_10k = revenues_df.loc[revenues_df['form'] == '10-K']

In [222]:
revenues_df_10k.head(3)

,start,end,val,accn,fy,fp,form,filed,frame
0,2007-01-01,2007-12-31,98786000000,0001047469-10-001151,2009,FY,10-K,2010-02-23,CY2007
5,2008-01-01,2008-12-31,103630000000,0001047469-10-001151,2009,FY,10-K,2010-02-23,NaN
6,2008-01-01,2008-12-31,103630000000,0001047469-11-001117,2010,FY,10-K,2011-02-22,CY2008


## Company Company Concepts

When querying for specific company concepts:

> *NOTE: this is easily attainable when using the previous query on the company facts*

In [195]:
acc_item = 'Revenues'

In [206]:
ibm_rev_req = requests.get(
                    f'https://data.sec.gov/api/xbrl/companyconcept/CIK{ibm_cik}/us-gaap/{acc_item}.json',
                    headers=headers
                    )

In [210]:
ibm_rev_dict = ibm_rev_req.json()['units']['USD']

In [211]:
ibm_rev_df = pd.DataFrame.from_dict(ibm_rev_dict)

In [217]:
ibm_rev_df.shape

(199, 9)

In [218]:
ibm_rev_df.head(3)

,start,end,val,accn,fy,fp,form,filed,frame
0,2007-01-01,2007-12-31,98786000000,0001047469-10-001151,2009,FY,10-K,2010-02-23,CY2007
1,2008-01-01,2008-06-30,51322000000,0001104659-09-045198,2009,Q2,10-Q,2009-07-28,NaN
2,2008-04-01,2008-06-30,26820000000,0001104659-09-045198,2009,Q2,10-Q,2009-07-28,CY2008Q2
